# 1. [Opt] Check GPU Type

In [ ]:
# Check GPU Type
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
    print('and then re-execute this cell.')
else:
    print(gpu_info)

Thu Dec  8 01:12:19 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    27W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 2. Git Clone from Github Repository to root directory
At Colab, location is "/content/".

Using `$ git clone https://github.com/lainyzine/git-clone.git` will load the Github repository with the name of the repository. With Colab, it can be convenient to work in the root directory. You can use the command to call the root directory as shown below.

According to the Github repository, we can use `! git pull origin main` or `! Just use git pull origin master`.

In [ ]:
# Clone from Github Repository
! git init .
! git remote add origin https://github.com/RichardMinsooGo-ML/Bible_3_21_Pytorch_YOLO_v3_Image.git
# ! git pull origin master
! git pull origin main

Initialized empty Git repository in /content/.git/
remote: Enumerating objects: 1948, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 1948 (delta 49), reused 2 (delta 2), pack-reused 1859
Receiving objects: 100% (1948/1948), 58.68 MiB | 17.40 MiB/s, done.
Resolving deltas: 100% (806/806), done.
From https://github.com/RichardMinsooGo-ML/Bible_3_16_Pytorch_YOLO_v3_Image
 * branch            main       -> FETCH_HEAD
 * [new branch]      main       -> origin/main


# 3. import clear_output

In [ ]:
from IPython.display import clear_output 
clear_output()

# 4. Data Engineering from VOC RAW Dataset

In [ ]:
'''
4. Data Engineering from VOC RAW Dataset
'''
!pip install --upgrade --no-cache-dir gdown

import gdown

!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar
# ! mkdir dataset
!tar -xvf VOCtrainval_11-May-2012.tar -C /content/dataset
! rm /content/VOCtrainval_11-May-2012.tar

clear_output()


# train valid list file generation
import os

str_train = open("tmp_train.txt", "w")
str_test  = open("tmp_val.txt", "w")

count = 0
for path, subdirs, files in os.walk(r"/content/dataset/VOCdevkit/VOC2012/Annotations"):
    for filename in files:
        f = os.path.join(filename)
        f = os.path.splitext(f)[0]
        
        count += 1
        if count%5 == 0:
            str_test.write("/content/dataset/VOC2012/images/" + str(f) +".jpg"+ os.linesep)
        else:
            str_train.write("/content/dataset/VOC2012/images/" + str(f) +".jpg"+ os.linesep)
        # str_train.write(str(f))
        
with open('tmp_train.txt') as infile, open('/content/dataset/VOC2012/train.txt', 'w') as outfile:
    for line in infile:
        if not line.strip(): continue  # skip the empty line
        outfile.write(line)  # non-empty line. Write it to output
with open('tmp_val.txt') as infile, open('/content/dataset/VOC2012/valid.txt', 'w') as outfile:
    for line in infile:
        if not line.strip(): continue  # skip the empty line
        outfile.write(line)  # non-empty line. Write it to output

! rm tmp_train.txt
! rm tmp_val.txt       

path = "/content/dataset/VOC2012/images/"
os.mkdir(path)
path = "/content/dataset/VOC2012/labels/"
os.mkdir(path)
path = "/content/dataset/VOC2012/xml_files/"
os.mkdir(path)

# move files 
import os
import shutil

source_folder = r"/content/dataset/VOCdevkit/VOC2012/JPEGImages//"
destination_folder = r"/content/dataset/VOC2012/images//"

# fetch all files
for file_name in os.listdir(source_folder):
    # construct full file path
    source = source_folder + file_name
    destination = destination_folder + file_name
    # move only files
    if os.path.isfile(source):
        shutil.move(source, destination)
        print('Moved:', file_name)

source_folder = r"/content/dataset/VOCdevkit/VOC2012/Annotations//"
destination_folder = r"/content/dataset/VOC2012/xml_files//"
# fetch all files
for file_name in os.listdir(source_folder):
    # construct full file path
    source = source_folder + file_name
    destination = destination_folder + file_name
    # move only files
    if os.path.isfile(source):
        shutil.move(source, destination)
        print('Moved:', file_name)

clear_output()

shutil.rmtree("/content/dataset/VOCdevkit")

# Create VOC Dataset form
"""
출처: https://github.com/pjreddie/darknet/blob/master/scripts/voc_label.py

Pascal VOC의 원본 annotation을 Yolo v3의 Object Detection label format으로 변환해주는 스크립트입니다.
이 파일을 VOCdevkit의 상위폴더로 옮긴 후, 해당 폴더에서 아래 명령을 실행하십시오.
    python voc_label.py

출력되는 파일은 아래와 같습니다.
    2007_test.txt
    train.txt
    voc_classes.txt
2007_test.txt, train.txt 파일을 열어보면 이미지들의 위치가 절대경로로 적혀있습니다.
만약 상대경로로 바꿔주려면 VSCode에서 알맞게 변환해줍니다.
"""

import os
import xml.etree.ElementTree as ET
import numpy as np

# classes = ["person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat", "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat", "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat", "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", "chair", "couch", "potted plant", "bed", "dining table", "toilet", "tv", "laptop", "mouse", "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"]

classes = ["aeroplane", "bicycle", "bird", "boat", "bottle", "bus", "car", "cat", "chair", "cow", "diningtable", "dog", "horse", "motorbike", "person", "pottedplant", "sheep", "sofa", "train", "tvmonitor"]

def convert(size, box):
    dw = 1. / (size[0])
    dh = 1. / (size[1])
    x = (box[0] + box[1]) / 2.0 - 1
    y = (box[2] + box[3]) / 2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return x, y, w, h

def convert_annotation(image_id):
    in_file  = open('/content/dataset/VOC2012/xml_files/%s.xml'%(image_id))
    out_file = open('/content/dataset/VOC2012/labels/%s.txt'%(image_id), 'w')
    # in_file  = open('COCO2017/Annotations/%s.xml'%(image_id))
    # out_file = open('COCO2017/labels/%s.txt'%(image_id), 'w')
    tree = ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)

    for obj in root.iter('object'):
        difficult = obj.find('difficult').text
        cls = obj.find('name').text
        if cls not in classes or int(difficult) == 1:
            continue
        cls_id = classes.index(cls)
        xmlbox = obj.find('bndbox')
        b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text),
             float(xmlbox.find('ymax').text))
        bb = convert((w, h), b)
        
        bb = np.around(bb, decimals=6)
        
        out_file.write(str(cls_id) + " " + " ".join([str(a) for a in bb]) + '\n')

sets = [('train'), ('valid')]

if __name__ == '__main__':
    wd = os.getcwd()

    for image_set in sets:
        image_ids = open('/content/dataset/VOC2012/%s.txt' % (image_set)).read().strip().split()
        for image_id in image_ids:
            
            image_id = image_id.split("/")
            image_id = image_id[5]
            image_id = image_id.split(".")
            image_id = image_id[0]
            
            # print(image_id)
            
            convert_annotation(image_id)
        # list_file.close()
          
clear_output()

# Zip Dataset 
# !zip -r /content/file.zip /content/dataset/VOC2012/labels

# 5. [Opt] Check Memory Space

In [ ]:
# Memory Space
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
    print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
    print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
    print('re-execute this cell.')
else:
    print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


# 6. install terminaltables for Ascii Table print

In [ ]:
!pip install terminaltables

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 7. Download Darknet Weights

In [ ]:
# Download Darknet Weights
# ! wget https://pjreddie.com/media/files/yolo-voc.weights 
! wget https://pjreddie.com/media/files/yolov3-tiny.weights 
! wget https://pjreddie.com/media/files/yolov3.weights
# ! wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights 
# ! wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.weights

import shutil
shutil.move("/content/yolov3-tiny.weights", "/content/checkpoints")
shutil.move("/content/yolov3.weights", "/content/checkpoints")
# shutil.move("/content/yolov4-tiny.weights", "/content/tmp")
# shutil.move("/content/yolov4.weights", "/content/tmp") 
shutil.rmtree("sample_data")


--2022-12-08 01:16:33--  https://pjreddie.com/media/files/yolov3-tiny.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35434956 (34M) [application/octet-stream]
Saving to: ‘yolov3-tiny.weights’

yolov3-tiny.weights 100%[===================>]  33.79M  15.6MB/s    in 2.2s    

2022-12-08 01:16:36 (15.6 MB/s) - ‘yolov3-tiny.weights’ saved [35434956/35434956]

--2022-12-08 01:16:36--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  23.1MB/s    in 11s     

2022-12-08 01:16:47 (21.5 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]



# 8. YOLO v3 Pre-Trained Weight gdown from Auther's GDrive

In [ ]:
import gdown

google_path = 'https://drive.google.com/uc?id='
file_id = '1ZOqdFdqSnHUd9X_TfBGktScjDTeeZ8gc'
output_name = 'Yolo_V3_VOC.pth'
gdown.download(google_path+file_id,output_name,quiet=False)
# https://drive.google.com/file/d/1ZOqdFdqSnHUd9X_TfBGktScjDTeeZ8gc/view?usp=sharing

shutil.move("/content/Yolo_V3_VOC.pth", "./checkpoints")

Downloading...
From: https://drive.google.com/uc?id=1ZOqdFdqSnHUd9X_TfBGktScjDTeeZ8gc
To: /content/Yolo_V3_VOC.pth
100%|██████████| 248M/248M [00:01<00:00, 149MB/s]


'./checkpoints/Yolo_V3_VOC.pth'

# 9. YOLO v3-tiny Pre-Trained Weight gdown from Auther's GDrive

In [ ]:
google_path = 'https://drive.google.com/uc?id='
file_id = '1hbz__T-j1Si2o6fGkjhYxRZRsvCTOnCH'
output_name = 'Yolo_V3_VOC_tiny.pth'
gdown.download(google_path+file_id,output_name,quiet=False)
# https://drive.google.com/file/d/1hbz__T-j1Si2o6fGkjhYxRZRsvCTOnCH/view?usp=sharing

shutil.move("/content/Yolo_V3_VOC_tiny.pth", "./checkpoints")

Downloading...
From: https://drive.google.com/uc?id=1hbz__T-j1Si2o6fGkjhYxRZRsvCTOnCH
To: /content/Yolo_V3_VOC_tiny.pth
100%|██████████| 35.5M/35.5M [00:01<00:00, 28.4MB/s]


'./checkpoints/Yolo_V3_VOC_tiny.pth'

In [ ]:
# from google.colab import drive 
# drive.mount('/content/gdrive/')

# 10. Train YOLO v3 from Pre-trained Weights

In [ ]:
! python train.py --num_epochs 3 --batch_size 8 --data_config config/VOC.data --model_def config/yolov3.cfg --pretrained_path checkpoints/Yolo_V3_VOC.pth --save_path checkpoints/Yolo_V3_VOC.pth

{'data_config': 'config/VOC.data', 'model_def': 'config/yolov3.cfg', 'pretrained_path': 'checkpoints/Yolo_V3_VOC.pth', 'save_path': 'checkpoints/Yolo_V3_VOC.pth', 'working_dir': './', 'num_epochs': 3, 'batch_size': 8, 'img_size': 416, 'n_cpu': 1, 'gradient_accumulations': 2, 'evaluation_interval': 2, 'multiscale_training': True, 'checkpoint_freq': 2, 'iou_thres': 0.5, 'conf_thres': 0.5, 'nms_thres': 0.5, 'ckpt_dir': './checkpoints', 'logs_dir': './logs'}
cuda
checkpoints/Yolo_V3_VOC.pth
Trained pytorch weight loaded!
1700
  4% 60/1700 [00:41<16:17,  1.68it/s]/content/utils/datasets.py:107: UserWarning: loadtxt: Empty input file: "/content/dataset/VOC2012/labels/2008_005145.txt"
  boxes = torch.from_numpy(np.loadtxt(label_path).reshape(-1, 5))
 12% 203/1700 [01:50<16:37,  1.50it/s]/content/utils/datasets.py:107: UserWarning: loadtxt: Empty input file: "/content/dataset/VOC2012/labels/2008_004172.txt"
  boxes = torch.from_numpy(np.loadtxt(label_path).reshape(-1, 5))
 23% 396/1700 [03:25<

# 11. Train YOLO v3 from Darknet Weights

In [ ]:
! python train.py --num_epochs 3 --batch_size 8 --data_config config/VOC.data --model_def config/yolov3.cfg --pretrained_path checkpoints/yolov3.weights --save_path checkpoints/Yolo_V3_VOC.pth

{'data_config': 'config/VOC.data', 'model_def': 'config/yolov3.cfg', 'pretrained_path': 'checkpoints/yolov3.weights', 'save_path': 'checkpoints/Yolo_V3_VOC.pth', 'working_dir': './', 'num_epochs': 3, 'batch_size': 8, 'img_size': 416, 'n_cpu': 1, 'gradient_accumulations': 2, 'evaluation_interval': 2, 'multiscale_training': True, 'checkpoint_freq': 2, 'iou_thres': 0.5, 'conf_thres': 0.5, 'nms_thres': 0.5, 'ckpt_dir': './checkpoints', 'logs_dir': './logs'}
cuda
checkpoints/yolov3.weights
Darknet weight loaded!
1700
 17% 287/1700 [02:10<16:21,  1.44it/s]/content/utils/datasets.py:107: UserWarning: loadtxt: Empty input file: "/content/dataset/VOC2012/labels/2008_004172.txt"
  boxes = torch.from_numpy(np.loadtxt(label_path).reshape(-1, 5))
 33% 565/1700 [04:30<08:40,  2.18it/s]
---- [Epoch 1/3, Batch 566/1700] ----
+------------+--------------+--------------+--------------+
| Metrics    | YOLO Layer 0 | YOLO Layer 1 | YOLO Layer 2 |
+------------+--------------+--------------+--------------+

# 12. Train YOLO v3-tiny from Pre-trained Weights

In [ ]:
! python train.py --num_epochs 3 --batch_size 32 --data_config config/VOC.data --model_def config/yolov3-tiny.cfg --pretrained_path checkpoints/Yolo_V3_VOC_tiny.pth --save_path checkpoints/Yolo_V3_VOC_tiny.pth

{'data_config': 'config/VOC.data', 'model_def': 'config/yolov3-tiny.cfg', 'pretrained_path': 'checkpoints/Yolo_V3_VOC_tiny.pth', 'save_path': 'checkpoints/Yolo_V3_VOC_tiny.pth', 'working_dir': './', 'num_epochs': 3, 'batch_size': 32, 'img_size': 416, 'n_cpu': 1, 'gradient_accumulations': 2, 'evaluation_interval': 2, 'multiscale_training': True, 'checkpoint_freq': 2, 'iou_thres': 0.5, 'conf_thres': 0.5, 'nms_thres': 0.5, 'ckpt_dir': './checkpoints', 'logs_dir': './logs'}
cuda
checkpoints/Yolo_V3_VOC_tiny.pth
Trained pytorch weight loaded!
425
  9% 37/425 [00:12<01:54,  3.40it/s]/content/utils/datasets.py:107: UserWarning: loadtxt: Empty input file: "/content/dataset/VOC2012/labels/2008_007355.txt"
  boxes = torch.from_numpy(np.loadtxt(label_path).reshape(-1, 5))
 33% 140/425 [00:43<01:16,  3.71it/s]
---- [Epoch 1/3, Batch 141/425] ----
+------------+--------------+--------------+
| Metrics    | YOLO Layer 0 | YOLO Layer 1 |
+------------+--------------+--------------+
| grid_size  | 13 

# 13. Train YOLO v3-tiny from Darknet Weights

In [ ]:
! python train.py --num_epochs 8 --batch_size 32 --data_config config/VOC.data --model_def config/yolov3-tiny.cfg --pretrained_path checkpoints/yolov3-tiny.weights --save_path checkpoints/Yolo_V3_VOC_tiny.pth

{'data_config': 'config/VOC.data', 'model_def': 'config/yolov3-tiny.cfg', 'pretrained_path': 'checkpoints/yolov3-tiny.weights', 'save_path': 'checkpoints/Yolo_V3_VOC_tiny.pth', 'working_dir': './', 'num_epochs': 8, 'batch_size': 32, 'img_size': 416, 'n_cpu': 1, 'gradient_accumulations': 2, 'evaluation_interval': 2, 'multiscale_training': True, 'checkpoint_freq': 2, 'iou_thres': 0.5, 'conf_thres': 0.5, 'nms_thres': 0.5, 'ckpt_dir': './checkpoints', 'logs_dir': './logs'}
cuda
checkpoints/yolov3-tiny.weights
Darknet weight loaded!
425
 33% 140/425 [00:42<01:23,  3.43it/s]
---- [Epoch 1/8, Batch 141/425] ----
+------------+--------------+--------------+
| Metrics    | YOLO Layer 0 | YOLO Layer 1 |
+------------+--------------+--------------+
| grid_size  | 12           | 24           |
| loss       | 2.895460     | 2.995627     |
| loss_x     | 0.067940     | 0.063433     |
| loss_y     | 0.039197     | 0.082050     |
| loss_w     | 0.157502     | 0.093578     |
| loss_h     | 0.096394    